In [2]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/akkirr/IT/annotated-diffusion


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from pathlib import Path

import torch
from torch import nan_to_num
from torchvision import transforms
from torch.utils.data import DataLoader

import numpy as np
from PIL import Image
import requests

In [41]:
from mylib import *

In [5]:
sampler = Sampler(linear_beta_schedule, 300)

In [6]:
# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# image = Image.open(requests.get(url, stream=True).raw) # PIL image of shape HWC
# image

In [7]:
# image_size = 128
# transform_aux = get_transform_aux(image_size)

# x_start = transform_aux(image).unsqueeze(0)
# x_start.shape

In [8]:
# t = torch.tensor([40])

# sampler.get_noisy_image(x_start, t)

In [9]:
# plot([sampler.get_noisy_image(x_start, torch.tensor([t])) for t in [0, 50, 100, 150, 199]])

In [10]:
from datasets import load_dataset

# load dataset from the hub
dataset = load_dataset("fashion_mnist")
image_size = 28
channels = 1
batch_size = 128

Found cached dataset fashion_mnist (/Users/akkirr/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# define image transformations (e.g. using torchvision)
transform = Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Lambda(lambda t: (t * 2) - 1)
])

# define function
def transforms(examples):
   examples["pixel_values"] = [transform(image.convert("L")) for image in examples["image"]]
   del examples["image"]

   return examples

transformed_dataset = dataset.with_transform(transforms).remove_columns("label")

# create dataloader
dataloader = DataLoader(transformed_dataset["train"], batch_size=batch_size, shuffle=True)

In [12]:
batch = next(iter(dataloader))
print(batch.keys())

dict_keys(['pixel_values'])


In [13]:
results_folder = Path("./results")
results_folder.mkdir(exist_ok = True)
save_and_sample_every = 1000

In [14]:
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Unet(
    dim=image_size,
    channels=channels,
    dim_mults=(1, 2, 4,)
)
model.to(device)

optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
from torchvision.utils import save_image

epochs = 6

for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
      optimizer.zero_grad()

      batch_size = batch["pixel_values"].shape[0]
      batch = batch["pixel_values"].to(device)

      # Algorithm 1 line 3: sample t uniformally for every example in the batch
      t = torch.randint(0, sampler.timesteps, (batch_size,), device=device).long()

      loss = sampler.p_losses(model, batch, t, loss_type="huber")

      if step % 100 == 0:
        print("Loss:", loss.item())

      loss.backward()
      optimizer.step()

      # save generated images
      if step != 0 and step % save_and_sample_every == 0:
        milestone = step // save_and_sample_every
        batches = num_to_groups(4, batch_size)
        all_images_list = list(map(lambda n: sampler.sample(model, batch_size=n, channels=channels), batches))
        all_images = torch.cat(all_images_list, dim=0)
        all_images = (all_images + 1) * 0.5
        save_image(all_images, str(results_folder / f'sample-{milestone}.png'), nrow = 6)

In [16]:
print(len(list(model.named_children())))

9


In [43]:
class Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.QKV = nn.Linear(1, 1)
        self.C = nn.Linear(1, 1)
        self.relu = nn.ReLU()

class TimeEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.time_proj = nn.Linear(1, 1)
        self.relu = nn.ReLU()

class LoraInjected(nn.Module):
    def __init__(self):
        super().__init__()
        self.src_linear = nn.Linear(1, 1)
        self.A = nn.Linear(1, 1)
        self.B = nn.Linear(1, 1)
        self.dropout = nn.Dropout1d()

class A(nn.Module):
    def __init__(self):
        super().__init__()
        self.just_linear = nn.Linear(1, 1)
        self.attn = Attention()
        self.time_embedder = TimeEmbedding()

a = A()

In [53]:
list(x[1] for x in find_modules(a, ["Attention"], (nn.Linear,), (LoraInjected,)))

['QKV', 'C']

In [55]:
x = nn.Linear(1, 1)

In [40]:
for name, mod in model.named_modules():
    print(name, mod.__class__.__name__)

 Unet
init_conv Conv2d
time_mlp Sequential
time_mlp.0 SinusoidalPositionEmbeddings
time_mlp.1 Linear
time_mlp.2 GELU
time_mlp.3 Linear
downs ModuleList
downs.0 ModuleList
downs.0.0 ResnetBlock
downs.0.0.mlp Sequential
downs.0.0.mlp.0 SiLU
downs.0.0.mlp.1 Linear
downs.0.0.block1 Block
downs.0.0.block1.proj WeightStandardizedConv2d
downs.0.0.block1.norm GroupNorm
downs.0.0.block1.act SiLU
downs.0.0.block2 Block
downs.0.0.block2.proj WeightStandardizedConv2d
downs.0.0.block2.norm GroupNorm
downs.0.0.block2.act SiLU
downs.0.0.res_conv Identity
downs.0.1 ResnetBlock
downs.0.1.mlp Sequential
downs.0.1.mlp.0 SiLU
downs.0.1.mlp.1 Linear
downs.0.1.block1 Block
downs.0.1.block1.proj WeightStandardizedConv2d
downs.0.1.block1.norm GroupNorm
downs.0.1.block1.act SiLU
downs.0.1.block2 Block
downs.0.1.block2.proj WeightStandardizedConv2d
downs.0.1.block2.norm GroupNorm
downs.0.1.block2.act SiLU
downs.0.1.res_conv Identity
downs.0.2 Residual
downs.0.2.fn PreNorm
downs.0.2.fn.fn LinearAttention
downs.0